In [1]:
import praw
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords

In [2]:
start = time.time()

In [3]:
#Stock Tickers and Names

URL = 'https://stockanalysis.com/stocks/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'lxml')
wsb_tickers = soup.find_all('li')

wsb_ticker_list = []
wsb_name_list = []

for i in wsb_tickers[12:-18]:
    #print(i.text.split(" - "))
    wsb_ticker_list.append(i.text.split(" - ")[0])
    wsb_name_list.append(i.text.split(" - ")[1])
    
len(wsb_ticker_list), len(wsb_name_list)

(6690, 6690)

In [4]:
#Crypto Tickers and Names

URL = 'https://coinmarketcap.com/all/views/all/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'lxml')
ssb_names = soup.find_all('td', 
                        class_ = 'cmc-table__cell cmc-table__cell--sticky cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__name')
ssb_name_list = []

for i in ssb_names:
    ssb_name_list.append(i.text.split('\">"')[0])

ssb_tickers = soup.find_all('td', 
                        class_ = 'cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol')
ssb_ticker_list = []

for i in ssb_tickers:
    ssb_ticker_list.append(i.text.split('\">"')[0])

len(ssb_ticker_list), len(ssb_name_list)

(200, 200)

In [5]:
reddit = praw.Reddit(client_id='HG7dA6CRLvCD_w',
                     client_secret='rH5FP42F__la6jpUdt01BQvZU48WiA', 
                     user_agent='WSB_Trends')

stopwords_list = stopwords.words('english')

for ind in range(len(stopwords_list)):
    stopwords_list[ind] = stopwords_list[ind].upper()

def comments_scraper(sub, comment_age, case_sensitive = False):
    posts = []
    subreddit = reddit.subreddit(sub)
    for post in subreddit.hot(limit = 25):
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

    comment_count = 0
    comments = ""

    for post_id in posts.id:
        submission = reddit.submission(id=post_id)
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            '''Loops through all comments. comment.body is a string with each comment's contents.
            comment.created is the time the comment was created.'''
            comment_age = (comment.created - time.time())/3600
            if comment_age <=comment_age:
                comments += comment.body + " "
                comment_count+=1
    if case_sensitive == False:
        comments = comments.upper()

    for character in'$ -.,\n':
        comments = comments.replace(character, " ")

    comments = comments.split()
    comments_counter = Counter(comments)
    return(comments_counter)

In [56]:
def wsb_leaderboard(n, hours):
    wsb_comments = comments_scraper('wallstreetbets', hours, case_sensitive = True)

    frequencies = []
    for tick in wsb_ticker_list:
        if tick not in stopwords_list:
            frequencies.append(wsb_comments[tick])
        else:
            frequencies.append(0)

    wsb_tickers_and_counts = pd.DataFrame([wsb_name_list, wsb_ticker_list, frequencies]).T
    wsb_tickers_and_counts.columns = ['stock','ticker','frequency']
    wsb_tickers_and_counts = wsb_tickers_and_counts.sort_values(by = ['frequency'], ascending = False)[0:n]
    wsb_tickers_and_counts.index = range(1,n+1)
    return wsb_tickers_and_counts

In [57]:
def ssb_leaderboard(n, hours):
    ssb_comments = comments_scraper('satoshistreetbets', hours, case_sensitive = True)

    frequencies = []

    for coin in range(len(ssb_name_list)):
        freq = 0
        #print(name_list[coin], ticker_list[coin])
        if ssb_name_list[coin] not in stopwords_list:
            freq+=ssb_comments[ssb_name_list[coin]]
        else:
            freq+=0
        if ssb_ticker_list[coin] not in stopwords_list:
            freq+=ssb_comments[ssb_ticker_list[coin]]
        else:
            freq+=0        

        frequencies.append(freq)

    ssb_tickers_and_counts = pd.DataFrame([ssb_name_list, ssb_ticker_list, frequencies]).T
    ssb_tickers_and_counts.columns = ['stock','ticker','frequency']
    ssb_tickers_and_counts = ssb_tickers_and_counts.sort_values(by = ['frequency'], ascending = False)[0:n]
    ssb_tickers_and_counts.index = range(1,n+1)
    return ssb_tickers_and_counts
    


In [58]:
wsb = wsb_leaderboard(10,6)

In [59]:
wsb

,stock,ticker,frequency
1,GameStop,GME,438
2,AMC Entertainment,AMC,95
3,Rocket Companies,RKT,82
4,UWM Holdings,UWMC,53
5,Palantir,PLTR,51
6,DuPont,DD,48
7,Restoration Hardware,RH,37
8,Tanger Factory Outlet Centers,SKT,22
9,Purple Innovation,PRPL,16
10,Tesla,TSLA,10


In [43]:
def string_form(df):
    string_form = "*Ticker, Name: Frequency*\n"
    for i in range(df.shape[0]):
        string_form += df.iloc[i].ticker +", "+ df.iloc[i].stock +": "+ str(df.iloc[i].frequency)+"\n"
    return(string_form)

In [45]:
print(string_form(wsb))


*Ticker, Name: Frequency*
GME, GameStop: 371
AMC, AMC Entertainment: 95
RKT, Rocket Companies: 82
RH, Restoration Hardware: 69
PLTR, Palantir: 57
UWMC, UWM Holdings: 50
DD, DuPont: 42
SKT, Tanger Factory Outlet Centers: 24
PRPL, Purple Innovation: 15
TD, Toronto-Dominion Bank: 14



In [9]:
ssb_leaderboard(10,6)

,name,ticker,frequency
4,Cardano,ADA,108
63,Enjin Coin,ENJ,79
0,Bitcoin,BTC,46
26,VeChain,VET,27
1,Ethereum,ETH,22
6,XRP,XRP,18
119,Harmony,ONE,17
90,Decentraland,MANA,17
60,BitTorrent,BTT,17
84,Chiliz,CHZ,16


In [10]:
end = time.time()
print("Done in", round((end - start), 2), "seconds.")

Done in 55.63 seconds.
